In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine=create_engine('sqlite:///data/chinook.db')

In [3]:
tracks=pd.read_sql_table('tracks',engine)

In [9]:
tracks.head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [6]:
genres=pd.read_sql_table('genres',engine)

In [10]:
genre_track=genres.merge(tracks[['GenreId','Milliseconds']],on='GenreId',how='left').drop('GenreId',axis=1)

In [11]:
genre_track.head()

,Name,Milliseconds
0,Rock,343719
1,Rock,342562
2,Rock,230619
3,Rock,252051
4,Rock,375418


In [12]:
genre_time=genre_track.groupby('Name')['Milliseconds'].mean()

In [14]:
pd.to_timedelta(genre_time,unit='ms').dt.floor('s').sort_values().sample(5)

Name
TV Shows           00:35:45
Comedy             00:26:25
Reggae             00:04:07
Sci Fi & Fantasy   00:48:31
Opera              00:02:54
Name: Milliseconds, dtype: timedelta64[ns]

In [15]:
cust=pd.read_sql_table('customers',engine,columns=['CustomerId','FirstName','LastName'])

In [16]:
invoice=pd.read_sql_table('invoices',engine,columns=['InvoiceId','CustomerId'])

In [17]:
ii=pd.read_sql_table('invoice_items',engine,columns=['InvoiceId','UnitPrice','Quantity'])

In [18]:
cust_inv=cust.merge(invoice,on='CustomerId').merge(ii,on='InvoiceId')

In [19]:
cust_inv.head()

,CustomerId,FirstName,LastName,InvoiceId,UnitPrice,Quantity
0,1,Luís,Gonçalves,98,1.99,1
1,1,Luís,Gonçalves,98,1.99,1
2,1,Luís,Gonçalves,121,0.99,1
3,1,Luís,Gonçalves,121,0.99,1
4,1,Luís,Gonçalves,121,0.99,1


In [20]:
total = cust_inv['Quantity'] * cust_inv['UnitPrice']

In [21]:
cols = ['CustomerId', 'FirstName', 'LastName']

In [23]:
cust_inv.assign(Total = total).groupby(cols)['Total'].sum().sort_values(ascending=False).head()

CustomerId  FirstName  LastName  
6           Helena     Holý          49.62
26          Richard    Cunningham    47.62
57          Luis       Rojas         46.62
46          Hugh       O'Reilly      45.62
45          Ladislav   Kovács        45.62
Name: Total, dtype: float64

In [24]:
# replicating previous operations with sql statement

In [25]:
sql_string1="""
select
    Name,
    time(avg(Milliseconds)/1000,'unixepoch') as avg_time
from (
        select
            g.Name
            ,t.Milliseconds
        from
            genres as g
        join
            tracks as t
            on
                g.genreid == t.genreid
    )
group by
    Name
order by
    avg_time
"""

In [27]:
pd.read_sql_query(sql_string1,engine).head()

,Name,avg_time
0,Rock And Roll,00:02:14
1,Opera,00:02:54
2,Hip Hop/Rap,00:02:58
3,Easy Listening,00:03:09
4,Bossa Nova,00:03:39
